In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
os.chdir("/home/sundari/MDM/Kaggle/Rossmann_Store_Sales")

In [ ]:
df_train = pd.read_csv("train.csv",sep=",")
df_stores = pd.read_csv("store.csv",sep=",")

In [ ]:
#Separate the stores running promo2 and not running
df_stores_promo2 = df_stores[df_stores['Promo2'] == 1]
df_stores_nopromo2 = df_stores[df_stores['Promo2'] == 0]

In [ ]:
#Find the mean distance in above two categories
stores_promo2 = df_stores_promo2['Store'].unique()
stores_nopromo2 = df_stores_nopromo2['Store'].unique()

np.mean(df_stores_promo2['CompetitionDistance']) #4316.508771929824
np.mean(df_stores_nopromo2['CompetitionDistance']) #6549.520295202952
np.std(df_stores_promo2['CompetitionDistance']) #5350.089645495236
np.std(df_stores_nopromo2['CompetitionDistance']) #9365.669297679668
#the mean compitetion distance is lower in case of stores running promo2, dig further to see how many stores 
#have compitetion and how many doesnt have in above two cases

len(df_stores_nopromo2.loc[df_stores_nopromo2['CompetitionDistance'].notnull() == False])
len(df_stores_promo2.loc[df_stores_promo2['CompetitionDistance'].notnull() == False])
(df_stores['CompetitionDistance'].notnull()).value_counts()
min(df_stores_nopromo2['CompetitionDistance'])

In [ ]:
df_merged = pd.merge(df_train,df_stores,on='Store',how='outer')
df_merged = df_merged[df_merged['Open'] == 1]

In [ ]:
df_merged_promo2 = df_merged[df_merged['Store'].isin(stores_promo2)]
df_merged_nopromo2 = df_merged[df_merged['Store'].isin(stores_nopromo2)]

In [ ]:
#find the mean sales across two categories
np.mean(df_merged_promo2['Sales']) #5423.71821838335
np.mean(df_merged_nopromo2['Sales']) #6124.710161387789

#looks misleading, find the means only when store is open
df_merged_promo2 = df_merged_promo2[df_merged_promo2['Open'] == 1]
df_merged_nopromo2 = df_merged_nopromo2[df_merged_nopromo2['Open'] == 1]

np.std(df_merged_promo2['Sales']) #6558.386062196469 #Max 41551 #median 6075.0 #sd 2725.194716522114
np.std(df_merged_nopromo2['Sales']) #7350.557935493625 #Max 38722 #median 6685.0 #sd 3394.266975826838

In [ ]:
#Analyze "Promo" parameter - compare stores running promo on a given day with not running promo
df_merged_promo = df_merged[df_merged['Promo'] == 1]
df_merged_nopromo = df_merged[df_merged['Promo'] == 0]
devices_promos = df_merged_promo['Store'].unique() #all stores ran promo atleast once

In [ ]:
df_merged.head()

In [ ]:
df_grouped_promo = df_merged_promo.reindex(columns = ['Store','Sales']).groupby('Store')
df_grouped_nopromo = df_merged_nopromo.reindex(columns = ['Store','Sales']).groupby('Store')

In [ ]:
temp_p = df_grouped_promo.mean()
temp_p.reset_index(level=0,inplace=True)
temp_p.columns = ['Store','Sales_Promo']
temp_np = df_grouped_nopromo.mean()
temp_np.reset_index(level=0,inplace=True)
temp_np.columns = ['Store','Sales_NoPromo']

In [ ]:
df_sales_comp = pd.merge(temp_p,temp_np,on='Store',how='outer')
df_sales_comp['Promo > NoPromo'] = (df_sales_comp['Sales_Promo'] > df_sales_comp['Sales_NoPromo'])
df_sales_comp['Promo > NoPromo'].value_counts()
#True     1114
#False       1

In [ ]:
df_merged.head()

In [ ]:
#Identify the monthly count of promotions
month_wise_count = df_merged.reindex(columns = ['Date'])
month_wise_count['Date'] = pd.to_datetime(month_wise_count['Date'])
month_wise_count['Month'] = month_wise_count['Date'].dt.month

In [ ]:
df_monthCount = month_wise_count.groupby('Month').count()
df_monthCount.reset_index(level=0,inplace=True)
df_monthCount.columns = ['Month','Num_Promos']

In [ ]:
df_stores['PromoInterval'].value_counts()

In [ ]:
df_merged_nopromo.head()

In [ ]:
#Find time since promo2
df_merged_promo2.head()

In [ ]:
df_merged_nopromo2['Promo'].value_counts() #234295,189012
df_merged_promo2['Promo'].value_counts() #233201,187884
#Promo2 and promo1 seem to be different
df_merged_promo2.head()

In [ ]:
df_merged_promo2['Date'] = pd.to_datetime(df_merged_promo2['Date'])
df_merged_promo2['Promo2SinceYear'] = pd.to_datetime(df_merged_promo2['Promo2SinceYear'].
                                                     astype('int').astype('str') + '-01-01')

In [ ]:
df_merged_promo2['Promo2_Since_years'] = (df_merged_promo2['Date'] - df_merged_promo2['Promo2SinceYear'])

In [ ]:
#pd.to_datetime(df_merged_promo2['Promo2SinceYear'].astype('int').astype('str') + '-01-01')
type(df_merged_promo2['Promo2SinceYear'].iloc[1])

In [ ]:
df_merged_promo2['Promo2_Since_years'].iloc[0]

In [ ]:
#Convert time delta to number
df_merged_promo2['Promo2_Since_years'] = (df_merged_promo2['Promo2_Since_years'] / np.timedelta64(1, 'D')).astype(int)
df_merged_promo2['Promo2_Since_years'] = df_merged_promo2['Promo2_Since_years']/365

In [ ]:
df_merged_promo2.head()

In [ ]:
df_merged_nopromo2['Promo2_Since_years'] = 0
#Completed adding a new parameter - Promo2_Since_years for two data frames, proceed

In [ ]:
df_merged = pd.concat([df_merged_promo2,df_merged_nopromo2])

In [ ]:
#Remove the promo2 parameters for now
df_merged.head()

In [ ]:
df_merged['Promo2_Since_years'] = np.where(df_merged['Promo2_Since_years'] < 0 ,0,df_merged['Promo2_Since_years'])

In [ ]:
#Remove unnecessary parameters
df_merged = df_merged.drop('DayOfWeek',axis=1)
df_merged = df_merged.drop('Customers',axis=1)
#df_merged = df_merged.drop('Date',axis=1)
df_merged = df_merged.drop('Promo2SinceWeek',axis=1)
df_merged = df_merged.drop('Promo2SinceYear',axis=1)
df_merged = df_merged.drop('PromoInterval',axis=1)
df_merged['CompetitionDistance'] = df_merged['CompetitionDistance'].fillna(0)

In [ ]:
#See if assortment and store type have impact on mean sales
#temp_asst_a = df_merged[df_merged['Assortment'] == 'a']
#temp_asst_b = df_merged[df_merged['Assortment'] == 'b']
#temp_asst_c = df_merged[df_merged['Assortment'] == 'c']
#temp_asst_d = df_merged[df_merged['Assortment'] == 'd']
df_merged.head()

In [ ]:
#Calculate time since compitetion the difference between Date and "compitetion since" parameters
#df_merged['CompetitionOpenSinceYear'] = df_merged['CompetitionOpenSinceYear'].fillna('1899')
#df_merged['CompetitionOpenSinceMonth'] = df_merged['CompetitionOpenSinceMonth'].fillna('1')

In [ ]:
# def convert_month( month ):
#     if(len(str(int(month))) > 1):
#         a = str(int(month)) + '-01'
#     else:
#         a = '0' + str(int(month)) + '-01'
#     return a;

In [ ]:
#df_merged['CompetitionOpenSinceMonth'] = df_merged['CompetitionOpenSinceMonth'].map(convert_month)
#df_merged['CompetitionOpenSinceYear'] = ((df_merged['CompetitionOpenSinceYear']).astype(int)).astype(str)

In [ ]:
#df_merged['CompitetionSince'] = df_merged[['CompetitionOpenSinceYear','CompetitionOpenSinceMonth']].apply(lambda x: '-'.join(x), axis=1)
#df_merged['CompitetionSince'] = pd.to_datetime(df_merged['CompitetionSince'])
#df_merged['Compitetion_Duration_Years'] = df_merged['Date'] - df_merged['CompitetionSince']
#df_merged = df_merged.drop('CompetitionOpenSinceYear', axis=1, inplace=True)
#df_merged = df_merged.drop('CompitetionSince', axis=1, inplace=True)
#df_merged = df_merged.drop('CompetitionOpenSinceMonth', axis=1, inplace=True)


In [ ]:
#Remove compitetion since for time being, fix it later
#df_merged = df_merged.drop('CompetitionOpenSinceYear',axis=1,inplace=True)

In [ ]:
df_merged.pop('CompetitionOpenSinceYear')
df_merged.pop('CompetitionOpenSinceMonth')
df_merged.pop('Date')
df_merged.pop('StateHoliday')

In [ ]:
df_merged.head()

In [ ]:
df_merged['StoreType'].value_counts()

In [ ]:
df_merged['Assortment'].unique()

In [ ]:
#Split data into train and test, use train data to build any models test 
from sklearn.cross_validation import train_test_split
df_train, df_test = train_test_split(df_merged, test_size = 0.3)
df_test['lookup'] = range(253318) #this is for remapping the data
df_train_bkp = df_train.copy(deep=True)
df_test_bkp = df_test.copy(deep=True)
    

In [ ]:
#Build models for different Assortment types
dummy_StoreType = pd.get_dummies(df_train['StoreType'])
df_train = df_train.drop('StoreType',axis=1)
dummy_StoreType.columns = ['StoreType_a','StoreType_b','StoreType_c','StoreType_d']
df_train = pd.concat([df_train,dummy_StoreType],axis=1)

In [ ]:
df_train_st_a = df_train[df_train['Assortment'] == 'a']
df_train_st_b = df_train[df_train['Assortment'] == 'b']
df_train_st_c = df_train[df_train['Assortment'] == 'c']

df_train_st_a.pop('Assortment')
df_train_st_b.pop('Assortment')
df_train_st_c.pop('Assortment')

df_train_st_a.pop('Store')
df_train_st_b.pop('Store')
df_train_st_c.pop('Store')

target_st_a = df_train_st_a['Sales']
target_st_b = df_train_st_b['Sales']
target_st_c = df_train_st_c['Sales']

df_train_st_a.pop('Sales')
df_train_st_b.pop('Sales')
df_train_st_c.pop('Sales')

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model_rf_st_a = RandomForestRegressor()
model_rf_st_b = RandomForestRegressor()
model_rf_st_c = RandomForestRegressor()
    
model_rf_st_a.fit(df_train_st_a,target_st_a)
model_rf_st_b.fit(df_train_st_b,target_st_b)
model_rf_st_c.fit(df_train_st_c,target_st_c)

In [ ]:
dummy_StoreType = pd.get_dummies(df_test['StoreType'])
df_test = df_test.drop('StoreType',axis=1)
dummy_StoreType.columns = ['StoreType_a','StoreType_b','StoreType_c','StoreType_d']
df_test = pd.concat([df_test,dummy_StoreType],axis=1)

In [ ]:
df_test_st_a = df_test[df_test['Assortment'] == 'a']
df_test_st_b = df_test[df_test['Assortment'] == 'b']
df_test_st_c = df_test[df_test['Assortment'] == 'c']

df_test_st_a.pop('Assortment')
df_test_st_b.pop('Assortment')
df_test_st_c.pop('Assortment')

df_test_st_a.pop('Store')
df_test_st_b.pop('Store')
df_test_st_c.pop('Store')

target_st_a = df_test_st_a['Sales']
target_st_b = df_test_st_b['Sales']
target_st_c = df_test_st_c['Sales']

df_test_st_a.pop('Sales')
df_test_st_b.pop('Sales')
df_test_st_c.pop('Sales')

lookup_st_a = df_test_st_a['lookup']
lookup_st_b = df_test_st_b['lookup']
lookup_st_c = df_test_st_c['lookup']

df_test_st_a.pop('lookup')
df_test_st_b.pop('lookup')
df_test_st_c.pop('lookup')

In [ ]:
predicted_st_a = model_rf_st_a.predict(df_test_st_a)
predicted_st_b = model_rf_st_b.predict(df_test_st_b)
predicted_st_c = model_rf_st_c.predict(df_test_st_c) 

In [ ]:
results_master = pd.DataFrame(df_test[['lookup','Sales']])
temp_st_a = pd.DataFrame(lookup_st_a)
temp_st_a['Predicted'] = predicted_st_a
temp_st_b = pd.DataFrame(lookup_st_b)
temp_st_b['Predicted'] = predicted_st_b
temp_st_b.head()
temp_st_c = pd.DataFrame(lookup_st_c)
temp_st_c['Predicted'] = predicted_st_c
temp_merged = pd.concat([temp_st_a,temp_st_b,temp_st_c])

In [ ]:
merged_results = pd.merge(results_master,temp_merged,on='lookup')
merged_results.columns = ['lookup', 'Sales', 'Predicted_AST']

In [ ]:
#retreive actual train and test backup
df_train = df_train_bkp.copy(deep=True)
df_test  = df_test_bkp.copy(deep=True)

#Build models for different store types types
dummy_StoreType = pd.get_dummies(df_train['Assortment'])
df_train = df_train.drop('Assortment',axis=1)
dummy_Assortment.columns = ['Assortment_a','Assortment_b','Assortment_c']
df_train = pd.concat([df_train,dummy_Assortment],axis=1)

In [ ]:
dummy_StoreType = pd.get_dummies(df_train['Assortment'])
df_train = df_train.drop('Assortment',axis=1)
dummy_Assortment.columns = ['Assortment_a','Assortment_b','Assortment_c']
df_train = pd.concat([df_train,dummy_Assortment],axis=1)

In [ ]:
df_train

In [ ]:
df_train_ast_a = df_train[df_train['StoreType'] == 'a']
df_train_ast_b = df_train[df_train['StoreType'] == 'b']
df_train_ast_c = df_train[df_train['StoreType'] == 'c']
df_train_ast_d = df_train[df_train['StoreType'] == 'd']

df_train_ast_a.pop('StoreType')
df_train_ast_b.pop('StoreType')
df_train_ast_c.pop('StoreType')
df_train_ast_d.pop('StoreType')

df_train_ast_a.pop('Store')
df_train_ast_b.pop('Store')
df_train_ast_c.pop('Store')
df_train_ast_d.pop('Store')

target_ast_a = df_train_ast_a['Sales']
target_ast_b = df_train_ast_b['Sales']
target_ast_c = df_train_ast_c['Sales']
target_ast_d = df_train_ast_d['Sales']

df_train_ast_a.pop('Sales')
df_train_ast_b.pop('Sales')
df_train_ast_c.pop('Sales')
df_train_ast_d.pop('Sales')

In [ ]:
df_train_ast_a.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model_rf_ast_a = RandomForestRegressor()
model_rf_ast_b = RandomForestRegressor()
model_rf_ast_c = RandomForestRegressor()
model_rf_ast_d = RandomForestRegressor()
    
model_rf_ast_a.fit(df_train_ast_a,target_ast_a)
model_rf_ast_b.fit(df_train_ast_b,target_ast_b)
model_rf_ast_c.fit(df_train_ast_c,target_ast_c)
model_rf_ast_d.fit(df_train_ast_d,target_ast_d)

In [ ]:
dummy_Assortment = pd.get_dummies(df_test['Assortment'])
df_test = df_test.drop('Assortment',axis=1)
dummy_Assortment.columns = ['Assortment_a','Assortment_b','Assortment_c']
df_test = pd.concat([df_test,dummy_Assortment],axis=1)

In [ ]:
df_test_ast_a = df_test[df_test['StoreType'] == 'a']
df_test_ast_b = df_test[df_test['StoreType'] == 'b']
df_test_ast_c = df_test[df_test['StoreType'] == 'c']
df_test_ast_d = df_test[df_test['StoreType'] == 'd']

df_test_ast_a.pop('StoreType')
df_test_ast_b.pop('StoreType')
df_test_ast_c.pop('StoreType')
df_test_ast_d.pop('StoreType')

df_test_ast_a.pop('Store')
df_test_ast_b.pop('Store')
df_test_ast_c.pop('Store')
df_test_ast_d.pop('Store')

target_ast_a = df_test_ast_a['Sales']
target_ast_b = df_test_ast_b['Sales']
target_ast_c = df_test_ast_c['Sales']
target_ast_d = df_test_ast_d['Sales']

df_test_ast_a.pop('Sales')
df_test_ast_b.pop('Sales')
df_test_ast_c.pop('Sales')
df_test_ast_d.pop('Sales')

In [ ]:
predicted_ast_a = model_rf_ast_a.predict(df_test_ast_a)
predicted_ast_b = model_rf_ast_b.predict(df_test_ast_b)
predicted_ast_c = model_rf_ast_c.predict(df_test_ast_c) 
predicted_ast_d = model_rf_ast_d.predict(df_test_ast_d)

In [ ]:
df_train.head()